In [10]:
import torch
import torchvision
import torch.nn as NeuralNet
import torch.optim as optim
import os
import copy
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [4]:
data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean, std)]),
                   'val': transforms.Compose([transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor(),
                                             transforms.Normalize(mean=mean, std=std)
                                             ]),
                  }

In [17]:
data_dir = 'Data/hymenoptera_data'
sets = ['train', 'val']
image_datasets = { x: datasets.ImageFolder(os.path.join(data_dir, x),
                                           data_transforms[x])
                    for x in ['train', 'val']}

data_loaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True)
                                                for x in ['train', 'val']}

data_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch} / {num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase =='train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(phase =='train'):
                    ouputs = model(inputs)
                    _, predictions = torch.max(ouputs, 1)
                    loss = criterion(ouputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(predictions == labels.data)
            
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = running_loss / data_sizes[phase]
            epoch_acc = running_corrects.double() / data_sizes[phase]


            print(f'{phase} Loss: {epoch_loss:.4f} Accuracy: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    time_elapsed = time.time() -since
    print(f'Training complete in { time_elapsed//60:.4f}m {time_elapsed%60:.0f}s')
    print(f'Best val accuracy: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

['ants', 'bees']


In [18]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = NeuralNet.Linear(num_ftrs, 2)
model.to(device)

criterion = NeuralNet.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

step_lr_scheduler = lr_scheduler.StepLR(optimizer=optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, 20)


c:\Users\Wakfu\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\Wakfu\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 / 19
----------
train Loss: 0.6919 Accuracy: 0.5082
val Loss: 0.5184 Accuracy: 0.7451

Epoch 1 / 19
----------
train Loss: 0.5465 Accuracy: 0.7377
val Loss: 0.3720 Accuracy: 0.9020

Epoch 2 / 19
----------
train Loss: 0.4862 Accuracy: 0.7664
val Loss: 0.3166 Accuracy: 0.9150

Epoch 3 / 19
----------
train Loss: 0.4002 Accuracy: 0.8197
val Loss: 0.2623 Accuracy: 0.9150

Epoch 4 / 19
----------
train Loss: 0.4093 Accuracy: 0.7951
val Loss: 0.2403 Accuracy: 0.9150

Epoch 5 / 19
----------
train Loss: 0.4231 Accuracy: 0.8074
val Loss: 0.2291 Accuracy: 0.9346

Epoch 6 / 19
----------
train Loss: 0.3883 Accuracy: 0.8115
val Loss: 0.2158 Accuracy: 0.9216

Epoch 7 / 19
----------
train Loss: 0.3744 Accuracy: 0.8361
val Loss: 0.2409 Accuracy: 0.9085

Epoch 8 / 19
----------
train Loss: 0.3185 Accuracy: 0.8689
val Loss: 0.2235 Accuracy: 0.9346

Epoch 9 / 19
----------
train Loss: 0.3327 Accuracy: 0.8525
val Loss: 0.2107 Accuracy: 0.9216

Epoch 10 / 19
----------
train Loss: 0.3327 Accura

In [19]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
num_ftrs = model.fc.in_features

model.fc = NeuralNet.Linear(num_ftrs, 2)
model.to(device)

criterion = NeuralNet.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

step_lr_scheduler = lr_scheduler.StepLR(optimizer=optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, 20)


c:\Users\Wakfu\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\Wakfu\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 / 19
----------
train Loss: 0.6988 Accuracy: 0.5738
val Loss: 0.6519 Accuracy: 0.6144

Epoch 1 / 19
----------
train Loss: 0.5812 Accuracy: 0.7172
val Loss: 0.4759 Accuracy: 0.7843

Epoch 2 / 19
----------
train Loss: 0.5503 Accuracy: 0.7008
val Loss: 0.4087 Accuracy: 0.8431

Epoch 3 / 19
----------
train Loss: 0.5114 Accuracy: 0.7582
val Loss: 0.3464 Accuracy: 0.8758

Epoch 4 / 19
----------
train Loss: 0.4044 Accuracy: 0.8607
val Loss: 0.3267 Accuracy: 0.8889

Epoch 5 / 19
----------
train Loss: 0.4464 Accuracy: 0.7746
val Loss: 0.3256 Accuracy: 0.8954

Epoch 6 / 19
----------
train Loss: 0.4322 Accuracy: 0.7869
val Loss: 0.2791 Accuracy: 0.9020

Epoch 7 / 19
----------
train Loss: 0.3848 Accuracy: 0.8566
val Loss: 0.2706 Accuracy: 0.9346

Epoch 8 / 19
----------
train Loss: 0.4158 Accuracy: 0.8115
val Loss: 0.2862 Accuracy: 0.9281

Epoch 9 / 19
----------
train Loss: 0.4053 Accuracy: 0.8279
val Loss: 0.2710 Accuracy: 0.9150

Epoch 10 / 19
----------
train Loss: 0.4272 Accura